# Exploratory Analysis and Visualization

This notebook contains some exploratory questions and analysis from the dataset

In [1]:
import pandas as pd
import seaborn as sns


In [5]:
# Import the dataset
df = pd.read_csv("../data/20240804_clean_data.csv.xz", compression="xz")
df.head()

,scientific_name,contig_acc,biosample_acc,target_acc,element_symbol,protein_acc,type,class,subclass,taxgroup_name,isolation_source,geo_loc_name,epi_type,host,collection_date,corrected_date,corrected_date2,year,country
0,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,PDT000041084.2,blaDHA,NaN,AMR,BETA-LACTAM,CEPHALOSPORIN,Salmonella enterica,NaN,USA,clinical,NaN,NaN,NaN,NaN,NaN,USA
1,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,PDT000041084.2,ble,EBK1426116.1,AMR,BLEOMYCIN,BLEOMYCIN,Salmonella enterica,NaN,USA,clinical,NaN,NaN,NaN,NaN,NaN,USA
2,Salmonella enterica,AAFUZC010000051.1,SAMN03098832,PDT000041084.2,blaNDM-1,EBK1426117.1,AMR,BETA-LACTAM,CARBAPENEM,Salmonella enterica,NaN,USA,clinical,NaN,NaN,NaN,NaN,NaN,USA
3,Salmonella enterica,AAFUZC010000073.1,SAMN03098832,PDT000041084.2,blaNDM-1,EBK1426163.1,AMR,BETA-LACTAM,CARBAPENEM,Salmonella enterica,NaN,USA,clinical,NaN,NaN,NaN,NaN,NaN,USA
4,Salmonella enterica,AAFUZC010000073.1,SAMN03098832,PDT000041084.2,ble,EBK1426164.1,AMR,BLEOMYCIN,BLEOMYCIN,Salmonella enterica,NaN,USA,clinical,NaN,NaN,NaN,NaN,NaN,USA
